# FETCH ARTICLES ABSTRACTS FROM SCOPUS SEARCH

## Miguel Brito


First you need an API KEY from Scopus, get it from https://dev.elsevier.com

In [20]:

API_KEY="dap9618fb94619c7600007661fff829e"


Your query in here

In [21]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [22]:
import requests
import json
import pandas as pd

In [23]:
def get_scopus_ids():
    resp = requests.get("http://api.elsevier.com/content/search/scopus?query=%s" % (QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    results= json.loads(resp.text.encode('utf-8'))
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

In [24]:
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

In [25]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

In [38]:
scopus_ids=get_scopus_ids()
print("number of scopus ids fetched : %d " % len(scopus_ids))
article_data=[]
n=0
for scopus_id in scopus_ids:
    results_info = get_scopus_info(scopus_id[0])
    results_text = get_scopus_text(scopus_id[0])
    title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
    pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
    try:
        abstract=results_text['full-text-retrieval-response']['coredata']['dc:description']
    except:
        abstract='(not available)'
    print("fetching %d..." % (n+1)) 
    n=n+1
    author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
    article_data.append({'Scopus_ID' : scopus_id[0],'title':title,'pubDate':pubDate,'abstract': abstract,'author': author[0][0]})
article_set=pd.DataFrame(article_data)

number of scopus ids fetched : 25 


In [39]:
article_set.head()
   

,Scopus_ID,abstract,author,pubDate,title
0,SCOPUS_ID:85035128299,Abstract\n \n T...,Lee M.,2018-06-01,Modeling and forecasting hotel room demand bas...
1,SCOPUS_ID:85038847134,(not available),Sevlian R.,2018-06-01,A scaling law for short term load forecasting ...
2,SCOPUS_ID:85036467319,Abstract\n \n C...,Kumar J.,2018-04-01,Workload prediction in cloud using artificial ...
3,SCOPUS_ID:85038816447,Abstract\n \n O...,Ma Y.,2018-04-01,Effects of user-provided photos on hotel revie...
4,SCOPUS_ID:85034053217,Abstract\n \n I...,Naik J.,2018-04-01,Short term wind power forecasting using hybrid...
